In [1]:
import pandas as pd
import numpy as np
import json
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.chrome.options import Options
import time
import os
import glob
from collections import defaultdict

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep

In [151]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


chrome_options = Options()
# turn off image loading
prefs = {"profile.managed_default_content_settings.images":2}
chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)

In [154]:
def button_click(n):
    all_resorts = []
    url = "https://www.tripadvisor.com/Search?q=ski+resort&queryParsed=true&searchSessionId#&ssrc=A&o=0"
    driver.get(url)
    button = driver.find_element_by_xpath('//a[@href="javascript:void(0);"][contains(text(),"Next")]')
    button.click()
    for i in range(1,n):
        button = driver.find_element_by_xpath('//a[@href="javascript:void(0);"][contains(text(),"Next")]')
        button.click()
        time.sleep(5)
        resorts = []
        url_location = "//div[contains(@class,'result_wrap')]"
        url_results = driver.find_elements_by_xpath(location)
        for url_result in url_results:
            dic = {}
            url_element = str(url_result.get_attribute('onclick'))
            if "Attraction_" in url_element:
                full_url = 'https://www.tripadvisor.com' + re.match(r".+(/Attraction_.+html).+",url_element).group(1)
                dic['url'] = full_url
                dic['resort_meta'] = url_result.text
                resorts.append(dic)
        df = pd.DataFrame.from_dict(resorts)
        df.to_csv(f'resorts_{i}.csv')
        print(i)

In [159]:
nums = np.arange(0,34)
dfs = []
for i in nums:
    file_name = f"resorts_{i}.csv"
    dfs.append(
        pd.read_csv(file_name)
    )

In [160]:
df = pd.concat(dfs).reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016 entries, 0 to 1015
Data columns (total 4 columns):
index          1016 non-null int64
Unnamed: 0     1016 non-null int64
resort_meta    1016 non-null object
url            1016 non-null object
dtypes: int64(2), object(2)
memory usage: 31.8+ KB


In [161]:
df.to_csv('all_resorts.csv')